# Instructions

Follow the steps contained in this notebook, which will contain instructions either to run a section of cells or to run another file contained in this folder.

Step 1: Create a database in postgres called `happiness_db`.  Using the Query Tool on this database, run the `schema.sql` file contained in this folder.

Step 2: Place a `config.py` in this folder and in it, define `username` as your postgres username and `password` as your postgres password.

Step 3: Run cells in this notebook until you reach Step 4.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

# Extract CSV's/Create Dataframes

In [2]:
happiness_file = "Resources/happiness2020.csv"
happiness_df = pd.read_csv(happiness_file)
happiness_df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [3]:
country_file = "Resources/countries.csv"
country_df = pd.read_csv(country_file,thousands=".",decimal=",")
country_df.head()

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


In [4]:
## ARH note: I added the thousands and decimals indicator to the read csv.

# Transform Happiness & Countries Data

In [6]:
happiness_df.replace({'Country name': {"Bosnia and Herzegovina" : "Bosnia & Herzegovina",
                                       "Congo (Kinshasa)" : "Congo DR",
                                       "Congo (Brazzaville)" : "Congo Republic",
                                       "Ivory Coast" : "Cote d'Ivoire",
                                       "Hong Kong S.A.R. of China" : "Hong Kong",
                                       "Taiwan Province of China" : "Taiwan",
                                       "Myanmar" : "Burma",
                                       "Trinidad and Tobago": "Trinidad & Tobago"}},
                                       inplace=True)

In [7]:
happiness = happiness_df.drop(columns= ["Regional indicator", "Ladder score", "Standard error of ladder score", 
                                        "upperwhisker", "lowerwhisker", "Ladder score in Dystopia", 
                                        "Perceptions of corruption", "Generosity", "Freedom to make life choices", 
                                        "Healthy life expectancy", "Social support", "Logged GDP per capita"],)
happiness.head()

,Country name,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [10]:
happiness = happiness.rename(columns={"Country name":"country",
                                            "Explained by: Log GDP per capita":"log_gdp",
                                            "Explained by: Social support":"social_support",
                                            "Explained by: Healthy life expectancy":"life_expectancy",
                                            "Explained by: Freedom to make life choices":"freedom_of_choice",
                                            "Explained by: Generosity":"generosity",
                                            "Explained by: Perceptions of corruption":"corruption_perception",
                                            "Dystopia + residual":"dystopia_residual"})
happiness.head()

,country,log_gdp,social_support,life_expectancy,freedom_of_choice,generosity,corruption_perception,dystopia_residual
0,Finland,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [11]:
country_df["Country"] = country_df["Country"].str.strip()
country_df.replace({'Country': {"Gambia, The" : "Gambia",
                                "Central African Rep." : "Central African Republic",
                                "Congo, Dem. Rep." : "Congo DR",
                                "Congo, Repub. of the" : "Congo Republic",
                                "Korea, South": "South Korea"}},inplace=True)

In [12]:
countries = country_df.drop(columns=['Region', 'Population', 'Area (sq. mi.)', 'Coastline (coast/area ratio)',
                'Phones (per 1000)', 'Arable (%)', 'Crops (%)', 'Other (%)', 'Climate', 'Birthrate', 
                 'Deathrate', 'Agriculture', 'Industry', 'Service'])
countries.head()

,Country,Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%)
0,Afghanistan,48.0,23.06,163.07,700.0,36.0
1,Albania,124.6,-4.93,21.52,4500.0,86.5
2,Algeria,13.8,-0.39,31.00,6000.0,70.0
3,American Samoa,290.4,-20.71,9.27,8000.0,97.0
4,Andorra,152.1,6.60,4.05,19000.0,100.0


In [13]:
countries = countries.rename(columns={'Country':'country', 'Pop. Density (per sq. mi.)':'pop_density', 
                                      'Net migration':'net_migration', 
                                      'Infant mortality (per 1000 births)':'infant_mortality', 
                                      'GDP ($ per capita)':'gdp', 'Literacy (%)':'literacy_rate'})
countries.head()

,country,pop_density,net_migration,infant_mortality,gdp,literacy_rate
0,Afghanistan,48.0,23.06,163.07,700.0,36.0
1,Albania,124.6,-4.93,21.52,4500.0,86.5
2,Algeria,13.8,-0.39,31.00,6000.0,70.0
3,American Samoa,290.4,-20.71,9.27,8000.0,97.0
4,Andorra,152.1,6.60,4.05,19000.0,100.0


### Previous cells outlining cleanup work, can probably delete these now?

In [14]:
# #Filter Dataframe
# happiness_columns = ["", "", ""]
# happiness_transformed= happiness_df[happiness_columns].copy()

# # Rename the column headers
# happiness_transformed = happiness_transformed.rename(columns={"": "",
#                                                           "": "",
#                                                           "": ""})

# # Clean the data by dropping duplicates and setting the index
# happiness_transformed.drop_duplicates("country", inplace=True)
# happiness_transformed.set_index("country", inplace=True)

# happiness_transformed.head()

In [15]:
# #Filter Dataframe
# country_columns = ["", "", ""]
# country_transformed= country_df[country_columns].copy()

# # Rename the column headers
# country_transformed = country_transformed.rename(columns={"": "",
#                                                           "": "",
#                                                           "": ""})

# # Clean the data by dropping duplicates and setting the index
# country_transformed.drop_duplicates("country", inplace=True)
# country_transformed.set_index("country", inplace=True)

# country_transformed.head()

# Create database connection/Load data into database

In [16]:
connection_string = f"{username}:{password}@localhost:5432/happiness_db"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
engine.table_names()

['countries', 'happiness']

In [20]:
happiness.to_sql(name='happiness', con=engine, if_exists='append', index=False)

In [24]:
countries.to_sql(name='countries', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from happiness', con=engine)

,country,log_gdp,social_support,life_expectancy,freedom_of_choice,generosity,corruption_perception,dystopia_residual
0,Finland,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266
...,...,...,...,...,...,...,...,...
148,Central African Republic,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265,2.860198
149,Rwanda,0.343243,0.522876,0.572383,0.604088,0.235705,0.485542,0.548445
150,Zimbabwe,0.425564,1.047835,0.375038,0.377405,0.151349,0.080929,0.841031
151,South Sudan,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157,1.378751


In [25]:
pd.read_sql_query('select * from countries', con=engine)

,country,pop_density,net_migration,infant_mortality,gdp,literacy_rate
0,Afghanistan,48.0,23.06,163.07,700.0,36.0
1,Albania,124.6,-4.93,21.52,4500.0,86.5
2,Algeria,13.8,-0.39,31.00,6000.0,70.0
3,American Samoa,290.4,-20.71,9.27,8000.0,97.0
4,Andorra,152.1,6.60,4.05,19000.0,100.0
...,...,...,...,...,...,...
222,West Bank,419.9,2.98,19.62,800.0,NaN
223,Western Sahara,1.0,NaN,NaN,NaN,NaN
224,Yemen,40.6,0.00,61.50,800.0,50.2
225,Zambia,15.3,0.00,88.29,800.0,80.6


Step 4: Return to the `happiness_db` in postgres, and using the Query Tool, run the section in `query.sql` labeled "Join tables on country."